In [104]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [105]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [106]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [107]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head(2)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
df.shape

(60000, 785)

In [109]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [110]:
X_train = X_train/255.0
X_test = X_test/255.0

In [111]:
class CustomData(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [112]:
train_dataset = CustomData(X_train, y_train)
test_dataset = CustomData(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)


In [113]:
class FirstCNN(nn.Module):
  def __init__(self, input_features):
    super().__init__()
    self.features = nn.Sequential(
      nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(32),
      nn.MaxPool2d(kernel_size=2, stride=2),

      nn.Conv2d(32, 64, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(64),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(64*7*7, 128),
      nn.ReLU(),
      nn.Dropout(0.4),

      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Dropout(0.4),

      nn.Linear(64,10)
    )

  def forward(self, x):
    x = self.features(x)
    x = self.classifier(x)
    return(x)



In [114]:
model = FirstCNN(1)
model.to(device)

epochs=100
learning_rate=0.01

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [115]:
for epoch in range(epochs):
  total_epoch_loss = 0 # Moved initialization outside the inner loop
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    #Forward Pass
    output = model(batch_features)
    #Calculate loss
    loss = criterion(output, batch_labels)
    #back pass
    optimizer.zero_grad()
    loss.backward()
    #Update Grads
    optimizer.step()
    total_epoch_loss = total_epoch_loss + loss.item()
  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 0.8232592675685882
Epoch: 2 , Loss: 0.6358217382580041
Epoch: 3 , Loss: 0.6352409969965617
Epoch: 4 , Loss: 0.6302050590564807
Epoch: 5 , Loss: 0.6413412532905738
Epoch: 6 , Loss: 0.6153806107242902
Epoch: 7 , Loss: 0.6273610559503238
Epoch: 8 , Loss: 0.6007430735131105
Epoch: 9 , Loss: 0.6479377845923106
Epoch: 10 , Loss: 0.6038189766407013
Epoch: 11 , Loss: 0.6179406471600135
Epoch: 12 , Loss: 0.6327582266926766
Epoch: 13 , Loss: 0.6172890062530836
Epoch: 14 , Loss: 0.5953046647409598
Epoch: 15 , Loss: 0.6239043084383011
Epoch: 16 , Loss: 0.5931886570851008
Epoch: 17 , Loss: 0.58955092638731
Epoch: 18 , Loss: 0.5972787314305703
Epoch: 19 , Loss: 0.601945077419281
Epoch: 20 , Loss: 0.5577265571157137
Epoch: 21 , Loss: 0.6378762739350399
Epoch: 22 , Loss: 0.5993847442318996
Epoch: 23 , Loss: 0.5700405199577411
Epoch: 24 , Loss: 0.6293846867978573
Epoch: 25 , Loss: 0.5889850558489561
Epoch: 26 , Loss: 0.5904618110756079
Epoch: 27 , Loss: 0.6219927667478721
Epoch: 28 , L

In [116]:
model.eval()

FirstCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [117]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8334166666666667


In [118]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8429375
